# This is the working notebook for the Telco Churn Project
   ### *Draft work and details can be found here*
   ### *Imports brought in as needed*

---

## **ACQUIRE**
- And examine overall data

In [1]:
#Common DS Libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

#Helper Functions
from acquire import get_telco_data

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

Created SQL Query that joined the customers with the word definitions of the "_id" columns.

In [2]:
df_raw = get_telco_data()
pd.set_option("display.max_columns", None)
df_raw.head(2)

,payment_type_id,internet_service_type_id,customer_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,signup_date,internet_service_type,payment_type
0,2,1,0002-ORFBO,2,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,2021-04-21 18:07:34,DSL,Mailed check
1,2,1,0003-MKNFE,1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,2021-04-21 18:07:34,DSL,Mailed check


In [ ]:
df_raw.info()

#### Upon investigation I noticed there were 11 records with zero tenure, who also had zero total charges.  I eliminated those records and converted total charges to a float.

In [ ]:
# NOTE: Commented out since this is reproduced in the Prepare Step.

# df_raw = df_raw[df_raw.total_charges != ' ']
# df_raw.total_charges = df_raw.total_charges.astype('float')
# df_raw.info()

#### Great - now no missing data!

#### Create seperate dfs with categorical and numerical only to get some general univariate info

In [ ]:
#create df with cat columns only
df_raw_cat = df_raw[[col for col in df_raw.columns if df_raw[col].dtype == 'object']]
#remove customer id since it has lots of options
df_raw_cat = df_raw_cat.drop(columns=['customer_id'])

In [ ]:
#value counts in cat columns
for col in df_raw_cat:
    print(df_raw_cat[col].value_counts())
    print('-----')

In [ ]:
#create df with only numerical columns
df_raw_num = df_raw[[col for col in df_raw.columns if df_raw[col].dtype != 'object']]
df_raw_num.head(2)

In [ ]:
#remove '-id' and date columns
df_raw_num = df_raw_num.drop(columns = ['payment_type_id','internet_service_type_id','contract_type_id','signup_date'])

In [ ]:
#get descriptive stats for each statistic
df_raw_num.describe().T

## DATA SUMMARY
- Univariate Analysis
- Note unnecessary data/data to remove or clean in prep stage
- Form initial hypothesees and plans

In [ ]:
from EDA_functions import explore_univariate

In [ ]:
#cat and num column names
df_raw_cat_vars = [col for col in df_raw_cat.columns]
df_raw_num_vars = [col for col in df_raw_num.columns]

In [ ]:
explore_univariate(df_raw,df_raw_cat_vars,df_raw_num_vars)

### Initial Dataset Analysis
- Gender and partner status are about even
- 2:1 Depedents (families) to none
- Vast majority have phone service (9:1)
- For those with phone lines, fairly even split between single and multiple
- Only 30% have online security
- 35% have online backup
- 35% also have device protection (note: explore device vs battery backup)
- Only 30% have tech support (price if signifigant)
- Even split for streaming for those with internet service
- Streaming movies mirrors streaming internet
- 60/40 paperless vs paper billing
- **26.59% overall churn rate**
- 55% month to month, almost evenly split between one year and two year
- 56/44 split between Fiber and DSL
- 1/3 pay by auto check, the rest evenly distributed between mailed, bank transfer and cc
- 16% senior citizens
- Tenure short and long, mean 32 months (mean without top and bottom slightly higher than this mean)
- Monthly charges normal with big low end mode

### Data Prep Plan
- Remove the 11 zero-tenure month records
- Drop customer id
- Drop all id categories (covered by actual categories which are just as wasy to encode)
- Drop signup date (since all churn dates = 1/2022, all tenure captures this information)
- Convert all yes/no to 1/0
- Convert total charges to float

### Questions to Examine
- Is there any difference between genders and partner status with churn (will be impactful since they are so evently represented)
- Any churn difference between families vs individuals?
- What are the impacts of having online security, online backup, device protection, tech support, streaming movies and streaming internet have on churn?  How do these variables relate to one another?
- What are the effects of internet service type, and how does that interplay with the above internet services (security, backup, etc. - multivariate analysis?)
- Does being a senior have an impact?  If not a huge impact shouldn't be a big deal since they are such a small percentage of overall users.
- Does payment method have an impact on churn?

## PREPARE

Prepare from file informed by above analysis.

In [3]:
from prepare import prep_telco
df = prep_telco(df_raw)
df.head(2)

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded,multiple_lines_No,multiple_lines_No phone service,multiple_lines_Yes,online_security_No,online_security_No internet service,online_security_Yes,online_backup_No,online_backup_No internet service,online_backup_Yes,device_protection_No,device_protection_No internet service,device_protection_Yes,tech_support_No,tech_support_No internet service,tech_support_Yes,streaming_tv_No,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No,streaming_movies_No internet service,streaming_movies_Yes,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1


### Split into train, validate and test datasets according to 56% Train, 24% Validate, 20% Test (random seed = 123)

In [4]:
from splitter import splitter
df_train, df_validate, df_test = splitter(df, target = 'churn')
df_train.sample(2)

Train = 56.0% | Validate = 24.0% | Test = 20.0%


,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded,multiple_lines_No,multiple_lines_No phone service,multiple_lines_Yes,online_security_No,online_security_No internet service,online_security_Yes,online_backup_No,online_backup_No internet service,online_backup_Yes,device_protection_No,device_protection_No internet service,device_protection_Yes,tech_support_No,tech_support_No internet service,tech_support_Yes,streaming_tv_No,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No,streaming_movies_No internet service,streaming_movies_Yes,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
4308,Male,0,No,No,38,Yes,Yes,No,Yes,Yes,No,No,Yes,Yes,95.1,3691.2,No,Month-to-month,Fiber optic,Credit card (automatic),0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0
5579,Male,0,No,No,1,No,No phone service,No,No,No,No,No,Yes,No,35.9,35.9,Yes,Month-to-month,DSL,Electronic check,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0


## Bivariate Analysis

In [ ]:
df_train.info()

### Create categorical and numerical lists.  For numerical remove all encoded variables.

In [ ]:
from EDA_functions import explore_bivariate
df_train_cat_vars = [col for col in df_train.columns if df_train[col].dtype == 'object']
df_train_num_vars = [col for col in df_train.columns if df_train[col].dtype != 'object'][1:4]

In [ ]:
df_train_cat_vars, df_train_num_vars

### Most variables are categorical, so using a bivariate exploration function to look at all of them vs churn to see which have an impact on churn.  

In [ ]:
from EDA_functions import explore_bivariate

#### Senior citizen was already 'encoded' as an integer and therefore did not appear in the categorical list pulled from the dataframe info, so seperately did this one:

In [ ]:
explore_bivariate(df_train,'churn_encoded',df_train_cat_vars,df_train_num_vars)

Followed up using this technique to determine impact of cat variables, as well as determine their independence (or lack thereof) from each other:

In [ ]:
import itertools
from scipy import stats

In [ ]:
explore_bivariate(df_train,'churn_encoded',['senior_citizen'])#,df_train_num_vars)

In [ ]:
combinations = list(itertools.combinations(df_train_cat_vars, 2))
combinations

In [ ]:
alpha = 0.05
outcomes = []
for combination in combinations:
    outcome = {}
    outcome["column1"] = combination[0]
    outcome["column2"] = combination[1]

    observed = pd.crosstab(df[combination[0]], df[combination[1]])
    
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    
    outcome["p"] = p
    
    if p < alpha:
        outcome["reject_null"] = True
    else:
        outcome["reject_null"] = False
    
    outcomes.append(outcome)

pd.set_option('display.max_rows', None)    
pd.DataFrame(outcomes)


In [ ]:
from EDA_functions import explore_bivariate_quant

In [ ]:
explore_bivariate_quant(df_train,'churn_encoded','monthly_charges')

#### Plot to examine the relationship between tenure, monthly charges and churn.

### Comparing monthly charges to churn using t-tests:

Seperate train set data into churned and not churned dfs

In [ ]:
train_churn_sample = df_train[df_train.churn == 'Yes']
train_nochurn_sample = df_train[df_train.churn == 'No']

One sided t-test against population mean:

In [ ]:
overall_mean = df_train.monthly_charges.mean()

t, p = stats.ttest_1samp(train_churn_sample.monthly_charges, overall_mean)
t, p/2

One sided t-test comparing churn vs non-churned means:

In [ ]:
t, p = stats.ttest_ind(train_churn_sample.monthly_charges, train_nochurn_sample.monthly_charges, equal_var=False)
t, p / 2

NOTE: Total charges not particularly useful because those who churn quickly have low total charges, but so do those who have long tenure of just phone services.

### Both t-test support the alternative hypothesis, that monthly charges do effect churn negatively.

##### As such, let's do some dataframe grouping of different variables to see how they effect cost.

## Multivariate Analysis

In [ ]:
sns.relplot(data=df_train, x='tenure', y='monthly_charges', hue='churn')

In [ ]:
for col in df_train_cat_vars:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df_train, x=col,y='monthly_charges',hue='churn')
    plt.show()

# Classification Analysis

In [5]:
# drop all non-numeric columns except churn, but drop churn_encoded as well
columns_to_drop = ['gender', 'partner', 'dependents',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'contract_type', 'internet_service_type', 'payment_type','churn_encoded']

In [8]:
df_train_encoded = df_train.drop(columns=columns_to_drop)
df_validate_encoded = df_validate.drop(columns=columns_to_drop)
df_test_encoded = df_test.drop(columns=columns_to_drop)

In [9]:
# Split dataframes into target and non-target sets for modeling
X_train = df_train_encoded.drop(columns=["churn"])
y_train = df_train_encoded.churn

X_validate = df_validate_encoded.drop(columns=["churn"])
y_validate = df_validate_encoded.churn

X_test = df_test_encoded.drop(columns=["churn"])
y_test = df_test_encoded.churn

In [25]:
# run baseline
baseline = y_train.mode()
# Produce a boolean array with True representing a match between the baseline prediction and reality
matches_baseline_prediction = (y_train == baseline.values[0])
baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

Baseline accuracy: 0.73


#### Models must have higher accuracy than 73% to be considered.

In [28]:
from matrix_result import matrix_result_2d
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [50]:
# decision tree with multiple hypers
# create, fit, loop, sneak in validate and subtract
tree_metrics = []

for i in range(1,10):
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

# Fit the model (on train)
    tree = tree.fit(X_train, y_train)
    in_sample = tree.score(X_train, y_train)
    y_pred = tree.predict(X_train)

# Add to matrix_results_2d which is the report we'll look at
    result = pd.DataFrame(confusion_matrix(y_train, y_pred))
    results = matrix_result_2d(result)
    results['depth'] = i

# Do a quick validation fit and score just to see how the change in each hyperparameter effects overfit
    out_of_sample = tree.score(X_validate, y_validate)
    results['fit'] = in_sample - out_of_sample
    
    tree_metrics.append(results)

df_tree_results = pd.DataFrame(tree_metrics).round(2).set_index('depth')
df_tree_results

,true_positive,false_positive,true_negative,false_negative,precision,recall,accuracy,f1_score,support_pos,support_neg,fit
depth,,,,,,,,,,,
1,2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
2,2383,388,658,508,0.86,0.82,0.77,0.84,2891,1046,0.03
3,2723,627,419,168,0.81,0.94,0.80,0.87,2891,1046,0.02
4,2624,505,541,267,0.84,0.91,0.80,0.87,2891,1046,0.03
5,2685,544,502,206,0.83,0.93,0.81,0.88,2891,1046,0.04
6,2569,396,650,322,0.87,0.89,0.82,0.88,2891,1046,0.05
7,2579,340,706,312,0.88,0.89,0.83,0.89,2891,1046,0.07
8,2579,282,764,312,0.90,0.89,0.85,0.90,2891,1046,0.10
9,2617,237,809,274,0.92,0.91,0.87,0.91,2891,1046,0.12


#### Depth = 4 optimizes for fit and precision.

In [51]:
# random forest with multiple hypers
from sklearn.ensemble import RandomForestClassifier
forest_metrics = []

for i in range(1,10):
    for j in range(10,0,-1):
        forest = RandomForestClassifier(max_depth=i, min_samples_leaf = j,random_state=123)
    
    # Fit the model (on train)
        forest = forest.fit(X_train, y_train)
        in_sample = forest.score(X_train, y_train)
        y_pred = forest.predict(X_train)

    # Add to matrix_results_2d which is the report we'll look at
        result = pd.DataFrame(confusion_matrix(y_train, y_pred))
        results = matrix_result_2d(result)
        results['depth,leaves'] = (i,j)

    # Do a quick validation fit and score just to see how the change in each hyperparameter effects overfit
        out_of_sample = forest.score(X_validate, y_validate)
        results['fit'] = in_sample - out_of_sample
    
        forest_metrics.append(results)

df_forest_results = pd.DataFrame(forest_metrics).round(2).set_index('depth,leaves')
pd.set_option("max_rows", None)
df_forest_results

,true_positive,false_positive,true_negative,false_negative,precision,recall,accuracy,f1_score,support_pos,support_neg,fit
"depth,leaves",,,,,,,,,,,
"(1, 10)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 9)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 8)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 7)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 6)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 5)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 4)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 3)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00
"(1, 2)",2891,1046,0,0,0.73,1.00,0.73,0.85,2891,1046,0.00


#### Fit and precision are optimized at depth = 7, leaves = 6.

In [52]:
from sklearn.neighbors import KNeighborsClassifier
# knn with multiple hypers
knn_metrics = []

for i in range(1,25):
    knn = KNeighborsClassifier(n_neighbors = i)

# Fit the model (on train)
    knn = knn.fit(X_train, y_train)
    in_sample = knn.score(X_train, y_train)
    y_pred = knn.predict(X_train)

# Add to matrix_results_2d which is the report we'll look at
    result = pd.DataFrame(confusion_matrix(y_train, y_pred))
    results = matrix_result_2d(result)
    results['neighbors'] = i

# Do a quick validation fit and score just to see how the change in each hyperparameter effects overfit
    out_of_sample = knn.score(X_validate, y_validate)
    results['fit'] = in_sample - out_of_sample
    
    knn_metrics.append(results)

df_knn_results = pd.DataFrame(knn_metrics).round(2).set_index('neighbors')
df_knn_results

,true_positive,false_positive,true_negative,false_negative,precision,recall,accuracy,f1_score,support_pos,support_neg,fit
neighbors,,,,,,,,,,,
1,2890,7,1039,1,1.00,1.00,1.00,1.00,2891,1046,0.29
2,2891,541,505,0,0.84,1.00,0.86,0.91,2891,1046,0.11
3,2686,354,692,205,0.88,0.93,0.86,0.91,2891,1046,0.11
4,2793,560,486,98,0.83,0.97,0.83,0.89,2891,1046,0.07
5,2664,429,617,227,0.86,0.92,0.83,0.89,2891,1046,0.07
6,2763,584,462,128,0.83,0.96,0.82,0.89,2891,1046,0.05
7,2681,502,544,210,0.84,0.93,0.82,0.88,2891,1046,0.04
8,2752,597,449,139,0.82,0.95,0.81,0.88,2891,1046,0.03
9,2703,541,505,188,0.83,0.93,0.81,0.88,2891,1046,0.04


#### 15 neighbors optimizes for precision and fit.

In [59]:
# log reg
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression()

logit = logit.fit(X_train, y_train)
in_sample = logit.score(X_train, y_train)
y_pred = logit.predict(X_train)

result = pd.DataFrame(confusion_matrix(y_train, y_pred))
results = matrix_result_2d(result, output='print')

True Positive = 2580
False Positive = 467
True Negative = 579
False Negative = 311
Precision = 0.85
Recall = 0.89
Accuracy = 0.80
F1 Score = 0.87
Support, Positive = 2891
Support, Negative = 1046


In [61]:
#test logit on out of sample
y_pred = logit.predict(X_validate)
result = pd.DataFrame(confusion_matrix(y_validate, y_pred))
results = matrix_result_2d(result, output='print')

True Positive = 1091
False Positive = 197
True Negative = 252
False Negative = 148
Precision = 0.85
Recall = 0.88
Accuracy = 0.80
F1 Score = 0.86
Support, Positive = 1239
Support, Negative = 449
